In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
                           rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## Door knob

In [4]:
from pkg.utils.code_scraps import *

In [5]:
gscene.show_pose(crob.home_pose)

In [6]:
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(0.5,0.5,0.5), rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=True, collision=True, color=(1,0,0,1))

In [7]:
knob_s = add_lever(pscene, knob)

### prepare planner

In [8]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [9]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

   Use a production WSGI server instead.
 * Debug mode: off


### Plan

In [10]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['knob', 'lever_grip']
('ONZERO', 'knob')


In [23]:
goal_nodes = [('TURNED','knob')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
try: 0 - ('ONZERO', 'knob')->('TURNED', 'knob')
try: 0 - ('ONZERO', 'knob')->('TURNED', 'knob')
try: 0 - ('ONZERO', 'knob')->('TURNED', 'knob')
result: 0 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try: 0 - ('ONZERO', 'knob')->('ONZERO', 'grip0')
result: 0 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try: 0 - ('ONZERO', 'knob')->('TURNED', 'knob')
result: 0 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try transition motion
result: 0 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try: 0 - ('ONZERO', 'knob')->('ONZERO', 'grip0')
try: 0 - ('ONZERO', 'knob')->('ONZERO', 'grip0')
try: 0 - ('ONZERO', 'knob')->('ONZERO', 'grip0')
try transition motion
try transition motion
try transition motion
transition motion tried: True
result: 0 - ('ONZERO', 'knob')->('ONZERO', 'grip0') = success
branching: 0->1 (0.24/30.0 s, steps/err: 61(134.871959686 ms)/0.00130055535747)
try: 1 - ('ONZERO', 'grip0')->('ONZERO', 'knob')
try: 0 - ('ONZERO', 'knob')->('TURNED', 'knob')
result

result: 13 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try: 14 - ('ONZERO', 'grip0')->('TURNED', 'grip0')
result: 0 - ('ONZERO', 'knob')->('ONZERO', 'grip0') = success
try: 0 - ('ONZERO', 'knob')->('ONZERO', 'grip0')
branching: 0->16 (0.68/30.0 s, steps/err: 73(284.425020218 ms)/0.00176664390986)
try: 4 - ('ONZERO', 'grip0')->('ONZERO', 'knob')
try: 15 - ('ONZERO', 'knob')->('ONZERO', 'grip0')
try transition motion
result: 0 - ('ONZERO', 'knob')->('ONZERO', 'grip0') = success
try transition motion
branching: 0->17 (0.7/30.0 s, steps/err: 78(414.149999619 ms)/0.00157405890121)
result: 15 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try: 18 - ('ONZERO', 'knob')->('TURNED', 'knob')
result: 14 - ('ONZERO', 'grip0')->('ONZERO', 'knob') = success
result: 4 - ('ONZERO', 'grip0')->('ONZERO', 'knob') = success
branching: 14->19 (0.71/30.0 s, steps/err: 1(29.5460224152 ms)/0)
branching: 4->18 (0.72/30.0 s, steps/err: 1(7.2820186615 ms)/0)
transition motion tried: True
try: 19 - ('ONZERO',

try: 33 - ('ONZERO', 'knob')->('TURNED', 'knob')
result: 32 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try: 3 - ('ONZERO', 'grip0')->('TURNED', 'grip0')
result: 33 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try: 26 - ('ONZERO', 'grip0')->('ONZERO', 'knob')
result: 26 - ('ONZERO', 'grip0')->('ONZERO', 'knob') = success
branching: 26->34 (1.06/30.0 s, steps/err: 1(6.15787506104 ms)/0)
try: 34 - ('ONZERO', 'knob')->('TURNED', 'knob')
result: 34 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
end
constrained motion tried: True
try: 18 - ('ONZERO', 'knob')->('TURNED', 'knob')
result: 3 - ('ONZERO', 'grip0')->('TURNED', 'grip0') = success
branching: 3->35 (1.1/30.0 s, steps/err: 80(723.500967026 ms)/4.5932542432e-05)
result: 18 - ('ONZERO', 'knob')->('TURNED', 'knob') = fail
try: 35 - ('TURNED', 'grip0')->('TURNED', 'knob')
try: 26 - ('ONZERO', 'grip0')->('TURNED', 'grip0')
end
constrained motion tried: True
result: 35 - ('TURNED', 'grip0')->('TURNED', 'knob') = fail
try: 36 - ('TU

result: 51 - ('TURNED', 'grip0')->('TURNED', 'knob') = fail
result: 50 - ('TURNED', 'grip0')->('TURNED', 'knob') = fail
Goal reached
try: 20 - ('ONZERO', 'grip0')->('TURNED', 'grip0')
try: 15 - ('ONZERO', 'knob')->('ONZERO', 'grip0')
try transition motion
Goal reached
end
constrained motion tried: True
try: 17 - ('ONZERO', 'grip0')->('TURNED', 'grip0')
transition motion tried: True
result: 7 - ('ONZERO', 'grip0')->('TURNED', 'grip0') = success
branching: 7->54 (1.52/30.0 s, steps/err: 80(642.189025879 ms)/1.69144150332e-05)
result: 33 - ('ONZERO', 'knob')->('ONZERO', 'grip0') = success
joint min
constrained motion tried: False
branching: 33->55 (1.53/30.0 s, steps/err: 69(117.697954178 ms)/0.00130886934597)
Motion Plan Failure
try: 54 - ('TURNED', 'grip0')->('TURNED', 'knob')
result: 26 - ('ONZERO', 'grip0')->('TURNED', 'grip0') = fail
end
transition motion tried: True
transition motion tried: True
======================= terminated 2: Stop called from other agent  (1.6/30.0) =========

result: 38 - ('TURNED', 'grip0')->('ONZERO', 'grip0') = success
branching: 38->63 (1.91/30.0 s, steps/err: 80(443.588018417 ms)/3.12520902837e-05)
======================= terminated 0: Stop called from other agent  (1.9/30.0) ===============================
end
end
constrained motion tried: True
constrained motion tried: True
result: 38 - ('TURNED', 'grip0')->('ONZERO', 'grip0') = success
result: 20 - ('ONZERO', 'grip0')->('TURNED', 'grip0') = success
branching: 20->65 (1.94/30.0 s, steps/err: 80(434.675931931 ms)/5.30661355795e-05)
branching: 38->64 (1.94/30.0 s, steps/err: 80(447.00884819 ms)/3.12520902837e-05)
======================= terminated 14: Stop called from other agent  (1.9/30.0) ===============================
======================= terminated 16: Stop called from other agent  (1.9/30.0) ===============================
end
constrained motion tried: True
result: 17 - ('ONZERO', 'grip0')->('TURNED', 'grip0') = success
branching: 17->66 (1.97/30.0 s, steps/err: 80(445.881128

In [ ]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

('ONZERO', 'knob')->('ONZERO', 'grip0')
('ONZERO', 'grip0')->('TURNED', 'grip0')


### push back

In [14]:
term_state = snode_schedule[-1].state
gscene.show_pose(term_state.Q)
print(pscene.subject_name_list)
print(term_state.node)

['knob', 'lever_grip']
('TURNED', 'knob')


In [15]:
goal_nodes = [("ONZERO",'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=False)

try: 0 - ('TURNED', 'knob')->('ONZERO', 'knob')
result: 0 - ('TURNED', 'knob')->('ONZERO', 'knob') = fail
try: 0 - ('TURNED', 'knob')->('ONZERO', 'knob')
result: 0 - ('TURNED', 'knob')->('ONZERO', 'knob') = fail
try: 0 - ('TURNED', 'knob')->('TURNED', 'grip0')
try transition motion
transition motion tried: True
result: 0 - ('TURNED', 'knob')->('TURNED', 'grip0') = success
branching: 0->1 (0.21/30.0 s, steps/err: 138(123.822927475 ms)/0.00138774704828)
try: 0 - ('TURNED', 'knob')->('ONZERO', 'knob')
result: 0 - ('TURNED', 'knob')->('ONZERO', 'knob') = fail
try: 0 - ('TURNED', 'knob')->('TURNED', 'grip0')
try transition motion
transition motion tried: True
result: 0 - ('TURNED', 'knob')->('TURNED', 'grip0') = success
branching: 0->2 (0.31/30.0 s, steps/err: 82(94.5830345154 ms)/0.00121614261541)
try: 0 - ('TURNED', 'knob')->('TURNED', 'grip0')
try transition motion
transition motion tried: True
result: 0 - ('TURNED', 'knob')->('TURNED', 'grip0') = success
branching: 0->3 (0.4/30.0 s, ste

result: 20 - ('TURNED', 'knob')->('ONZERO', 'knob') = fail
try: 14 - ('ONZERO', 'grip0')->('ONZERO', 'knob')
result: 14 - ('ONZERO', 'grip0')->('ONZERO', 'knob') = fail
try: 10 - ('TURNED', 'knob')->('ONZERO', 'knob')
result: 10 - ('TURNED', 'knob')->('ONZERO', 'knob') = fail
try: 4 - ('TURNED', 'knob')->('ONZERO', 'knob')
result: 4 - ('TURNED', 'knob')->('ONZERO', 'knob') = fail
try: 3 - ('TURNED', 'grip0')->('TURNED', 'knob')
result: 3 - ('TURNED', 'grip0')->('TURNED', 'knob') = success
branching: 3->21 (2.72/30.0 s, steps/err: 1(3.88383865356 ms)/0)
try: 21 - ('TURNED', 'knob')->('ONZERO', 'knob')
result: 21 - ('TURNED', 'knob')->('ONZERO', 'knob') = fail
try: 13 - ('TURNED', 'grip0')->('ONZERO', 'grip0')
joint max
constrained motion tried: False
Motion Plan Failure
result: 13 - ('TURNED', 'grip0')->('ONZERO', 'grip0') = fail
try: 14 - ('ONZERO', 'grip0')->('ONZERO', 'knob')
result: 14 - ('ONZERO', 'grip0')->('ONZERO', 'knob') = fail
try: 14 - ('ONZERO', 'grip0')->('ONZERO', 'knob')

In [16]:
snode_schedule = tplan.get_best_schedule(home_pose=crob.home_pose)
ppline.play_schedule(snode_schedule)

('TURNED', 'knob')->('TURNED', 'grip0')
('TURNED', 'grip0')->('ONZERO', 'grip0')
('ONZERO', 'grip0')->('ONZERO', 'knob')
('ONZERO', 'knob')->('ONZERO', 'knob')
